## Sequência de funções valor para os credores

Queremos resolver por indução retroativa um jogo com T períodos.

Estou trabalhando em como deixar esse jogo em torno do parâmetro T

Teremos duas sequências de funções valor: uma para quando apenas o credor sênior propõe e outra para quando apenas o credor júnior propõe. As funções do jogo final serão uma média ponderada destas.



In [327]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [328]:
#libraries
import numpy as np
from numba import jit, jitclass, float64, njit
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from scipy.stats import beta

from random import uniform #para a draw da uniforme(0,1)
import math

import bisect
from bisect import bisect_left #para discretizar beta

### Passo 0: definindo comandos do latex para facilitar a escrita

$%conjunto de informações de s em t$
$\newcommand{\Is}[1]{\theta_{s{#1}}, l_{s{#1}}, l_{j{#1}}}$
$%macros para facilitar a escrita de funções valor$


$%conjunto de informações de j em t$
$\newcommand{\Ij}[1]{\theta_{j{#1}}, l_{s{#1}}, l_{j{#1}}}$

$%função valor de s em t$
$\newcommand{\Ws}[1]{ W_{s{#1}} ( \Is{{#1}}) }$

$%função valor de j em t$
$\newcommand{\Wj}[1]{ W_{j{#1}} ( \Ij{{#1}}) }$

$%operador esperança de s em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \mid ( \Is{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b de big para aumentar os parênteses$
$\newcommand{\esb}[2]{\mathbb{E_{#1}^{s} \bigg[ {#2}  \bigg] }}$

$%operador esperança de j em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \mid ( \Ij{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b para aumentar os parênteses$
$\newcommand{\ejb}[2]{\mathbb{E_{#1}^{j} \bigg[ {#2}  \bigg] }}$


$%comando para usar o máximo com chaves grandes$
$\newcommand{\maximo}[1]{\max \bigg\{ #1 \bigg\}}$


In [329]:
#parâmetros do jogo, apenas para ilustração a princípio



μ = 3 #número de meses entre períodos, não entendi onde entra ainda
ρ = 0.9 # (1 - ρ) é a taxa de depreciação da empresa a cada período
β = 1 #usamos aqui a distribuição Uniforme, depois vamos mudar isto
c0 = 0.05 #custo fixo de ir para a corte
c1 = 0.015 #custo variável de ir para a corte




λj = 0.5 #probabilidade de j propor a cada turno. Não precisaremos estimar, isso virá dos dados


θ_s0 = 0.5 #habilidade inicial de s
θ_j0 = 0.5 #habilidade inicial de j

In [330]:
#informações que virão dos dados

Vmax = 100
L = 20 
L_s = 10
L_j = 10

#valores da dívida de cada jogador (virá dos dados, aqui é exemplo):
Ds = 20
Dj = 10

### Qual o ganho que o credor s tem em propor a cada período?

Suponha que ele vai propor reorganizar a firma no perído t. A reorganização demora 1 período, então o valor da firma aparece no próximo período, $V_{t+1}$. E ele precisa fazer uma estimativa sobre $\theta_{s,t+1}$ dada a informação que ele tem no período atual, que eu chamei de k: $\theta_{s,k}$:

$$ R_t (\theta_{s,k}) = V_{t+1} \bigg( \mathbb{E}_k^s \big[ \theta_{s,t+1} \mid \theta_{s,k} \big] \bigg) - L_j$$


#### Qual a estimativa da habilidade dele nos períodos à frente dada a informação dele hoje?

Essa parte depende da hipótese sobre a distribuição da habilidade.

Se supormos que a habilidade segue distribuição Uniforme, a forma fechada dela é:

Suponha $ t > k$

$$ \mathbb{E}_k^s \big[ \theta_{s,t} \mid \theta_{s,k} \big] =  \frac{\theta_{s,k}} {2^{t - k}} + \sum_{n=1}^{t - k} \frac{1}{2^{n}} $$


#### Quais os passos do algoritmo para implementar essa estratégia?

1. Calcular todos os valores de reorganização $ R_t $ de $ t \in  \{k, k+1, ..., T\}$ e também o valor de nunca reorganizar, que é $L_s$

2. Criar uma condição para checar se $\max \{R_{k}, R_{k+1}, ..., R_{T}, L_s \} = R_{k}$
    * em caso positivo, oferecer $P_s = W_{j, t+1} = L_j$ a fim de reorganizar a firma
    * em caso negativo, oferecer $P_s < W_{j, t+1} = L_j$ a fim de adiantar o jogo até o próximo período
    
3. Repetir os passos 1 e 2 até que reorganize a firma ou o jogo termine com ela liquidada

### Passo 1: número máximo de turnos

Calculado com base nos parãmetros

In [331]:
#função para calcular o máximo de turnos do jogo
def maximo_de_turnos(ρ, Vmax, L):
    
    t = 1
    
    while ρ**t * Vmax > L:
        
        t = t + 1
        
    return t
    

In [332]:
#guardando o número máxim de turnos

T = maximo_de_turnos(ρ, Vmax, L)

T

16

In [333]:
#valor máximo de reorganização da firma a cada período


#sequência de valores da firma para cada período

V = np.empty(T, dtype=np.int_)

for t in range(T):

    V[t] = ρ**t * Vmax

In [334]:
#valor de liquidação


#a dívida total é sempre a soma das dívidas
D = Ds + Dj

#o custo total é uma função do tempo

def C(t):
    Ct = c0 * D + c1 * t * D
    
    return Ct

#e os valores de liquidação também são função do tempo

def Ls(t):
    
    Lst = min(L - C(t), Ds)
    
    return Lst

def Lj(t):
    
    Ljt = min(L - C(t) - Ls(t), Dj)
    
    return Ljt
    
    


In [335]:
V[0] #é o primeiro valor do negócio

V[15]

100

20

In [336]:
Ls(2)

17.6

### Definir os arrays para guardar as funções valor de cada período

São:

* 1000 slots para habilidade do jogador

* 1000 slots para o lower bound do adversário

* 2 slots para o lower bound do próprio jogador no próximo período
    * um para quando ele propôs no período anterior
    * outro para quando ele respondeu no período anterior
    
* T slots para marcar o período da função valor



In [337]:
#slots para cada habilidade
grid_size = 100


#a vantagem de colocar os dados assim é que se eu quiser teta_s = 0.115, basta procurar θs_vals[114]
θs_vals = np.linspace(0.01, 1, grid_size) 
θj_vals  = np.linspace(0.01, 1, grid_size)  

In [338]:
θs_vals[99 - 1]

0.99

In [339]:
#arrays para o credor sênior



# primeira entrada é a habilidade verdadeira do jogador
# segunda entrada é lst
# terceira entrada é ljt

# quarta entrada indica se jogador está propondo (1) ou respondendo (0)
# quinta entrada indica se é a função valor (0) ou se é a policy (1)
# última entrada é o período do jogo

Ws_array = np.zeros((100, 100, 100, 2,2, T))

Wj_array = np.zeros((100, 100, 100, 2,2, T))



In [340]:
#uma das sugestões que encontrei no stack overflow foi de dividir a matriz grandona em várias matrizes pequenas
#vou tentar com 100x100x100 primeiro, depois eu adapto o código

# Ws_array1 = np.empty((100, 100, 100,T))
# Ws_array2 = np.empty((100, 100, 100,T))
# Ws_array3 = np.empty((100, 100, 100,T))
# Ws_array4 = np.empty((100, 100, 100,T))
# Ws_array5 = np.empty((100, 100, 100,T))
# Ws_array6 = np.empty((100, 100, 100,T))
# Ws_array7 = np.empty((100, 100, 100,T))
# Ws_array8 = np.empty((100, 100, 100,T))
# Ws_array9 = np.empty((100, 100, 100,T))


Populando o array com os valores de liquidação, que independem das habilidades ou lower bounds



In [341]:
Ws_array[:,:,:, :, : (T-1)] = Ls(T)

Wj_array[:,:,:, :, : (T-1)] = Lj(T)

### Função para tirar os valores esperados de t períodos à frente

Essa é a base do jogo. Vamos começar com a Uniforme mesmo por ser mais fácil de computar (e verificar), depois eu mudo o código para considerar a Beta.

In [342]:
#função para calcular a esperança da habilidade nos períodos à frente
def expec(h, k, t):
    #h é a informação hoje, usada para prever a habilidade
    #k é o período de hoje
    #t é o período para o qual eu desejo estimar a habilidade
    
    #somatório de n até t-k
    summation = 0
    for i in range(1, t-k+1):      
        summation += 2**-i
    
    resultado = h/2**(t-k) + summation
    
    return round(resultado,2) #arredonda para duas casas decimais para ficar de acordo com o tamanho da matriz
    
# expec(0.5, 1, 4) #ok com os meus cálculos

#como ficaria um gráfico dessa função?


Como é a notação dessa matriz, na prática?


$ W_{st} (\Is{1})   =$ Ws_array[ $\theta_{st}, l_{st}, l_{jt}, t$] 

Só temos duas entradas para $l_{st}$ porque:

* ou $l_{st} = \theta_{s,t-1}$, caso s tenha proposto no período anterior

Neste caso, $l_{st}$ será igual à habilidade da entrada de $\theta_{st}$ correspondente

* ou $l_{st} = \maximo{l_{s,t-1}, c_{st}} $ caso s não tenha proposto no período anterior

Neste caso, precisamos calcular cst para ver qual é maior. 

### Populando os arrays nos períodos T-1, T-2, ..., 1, 0.








In [343]:
#funções para achar os valores nas matrizes

def Ws_find(teta_st, lst, ljt, proposer, policy, t):
    
    
    #para achar teta_st = 0.99, buscamos: θs_vals[99 - 1]
    #como os valores serão em casas decimais, temos que multiplicar por 100
    vteta_st = int(100 * teta_st - 1)
    
    vlst = int(100 * lst - 1)
    
    vljt = int(100 * ljt - 1)
    
    return Ws_array[vteta_st, vlst, vljt, proposer, policy, t]
    
    

    
def Wj_find(teta_jt, lst, ljt, proposer, policy, t):
    
    
    vteta_jt = int(100 * teta_jt - 1)
    
    vlst = int(100 * lst - 1)
    
    vljt = int(100 * ljt - 1)
    
    return Ws_array[vteta_jt, vlst, vljt, proposer, policy, t]

Quando S propõe



In [344]:
#payoff de S quanto ele tem o direito de propor em t

#aqui eu colo Wj[t] e Ws[t] como argumentos. 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def St(teta_st, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje pois s revela esta quando propõe
    
    ls_next = teta_st
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    
    
    #quanto s pagará para j neste período? A estimtativa da função valor de j amanhã, conforme as informações de j hoje.
    
    Pst = λj  * Wj_find(ljt, lst, ljt, 1, 0, t) + (1 - λj) * Wj_find(ljt, lst, ljt, 0, 0, t)
    
    
    while (Pst - Wj_find(cjt, ls_next, cjt, 0, 0, t) > tol and flag < 10000):
        cjt = cjt + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    #esperança da própria habilidade no próximo período
    θs_next = expec(teta_st, t, (t+1))
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    St = [Ls(t), V[t] * θs_next - Pst, Ws_find(θs_next, ls_next, lj_next, 1,0, t)]
          
    
    policy_St = np.argmax(St)
    
    
    #retorna um vetor com dois elementos: a função valor e a política ótima
    return St[policy_St], policy_St


    

    


In [345]:


Ws_find(0.5, 0.4, 0.5, 0, 0, 15)

11.3

In [346]:
St(0.5, 0.4, 0.5, T-1)[0]

11.75

In [347]:
Ws_array[50, 40, 50, 0, 0, T-1]

11.3

### Passo 2: definir as funções valor de liquidação e as matrizes para guardar seus valores


$$ W_{s1} (\Is{1}) = (1 - \lambda_j) S_1 (\Is{1}) + (\lambda_j) s_1 (\Is{1}) $$

A primeira parte da função valor de s é caso ele venha a propor:


 $$ S_1 (\Is{1}) = \max \bigg\{ L_{s1}, V \es{1}{\theta_s} - \es{1}{W_j}, \es{1}{W_s}
                    \bigg\} $$



A segunda parte da função valor é se se for chamado a responder:
    
$s_1 (\Is{1}) = $
    $ \esb{1}{ Prob ({j liquidar})  \maximo{L_{s1}, V \es{1}{\theta_s} - L_{j1}} } $
    $ + \esb{1}{ Prob ({j reorganizar})   \maximo{\ej{1}{W_s}, \es{1}{W_s}}    }$
 
 
 Sendo que
 
 $ \Es{1}{Prob (j liq em T-1)}$
 
  $$ = Prob \bigg(L_{j1} > \max \{V \es{1}{ \ej{1}{\theta_j} } - \es{1}{ \ej{1}{W_s} }, \es{1}{ \ej{1}{W_j}  }\} \bigg)$$
  
  
 Repare que $L_{j1} > L{j}$ e que $\es{1}{\theta_{j}}$ cresce em $\theta_{j}$. Então deve existir um threshold de reorganização em t = T-1, $\phi_{j1}$, tal que: 

$$V \phi_{j1} - L_{s} = L{j1} $$

$$ \phi_{j1} = \frac{ L_{j1} + L_{s} }{V} $$

Então se $\es{1}{\theta_{j}} > \phi_{j1}$, j reorganiza a firma. E j liquida a firma caso contrário.


A pergunta é: qual a probabilidade de que $\theta_{j}$ tome um valor menor ou igual a $\phi_{j1}$ ? Isso pode ser respondido usando a \textit{prior} do jogador, a função CDF da distribuição Beta:

$$ \Es{1}{Prob (\textnormal{j liq em T-1})} = F_{\beta} \big( \phi_{j1} \mid l_{j1} \big) $$
 


Agora o valor esperado caso j liquide. No paper isto é 

$  \maximo{L_{s1}, V \es{1}{\theta_s} - L_{j1}}  $

Repare que s quer prever a habilidade dele próprio no período final, então podemos usar a habilidade dele no período T-1 para isso

Agora, o payoff esperado de s caso j decida reorganizar a firma

$ \maximo{\ej{1}{W_s}, \es{1}{W_s}}    $

### Criando as funções valor para J


É análogo ao s, então vou colocar só a descrição geral aqui. Basicamente, é só trocar s por j nos argumentos

$$ W_{jt} (\Ij{t}) = (1 - \lambda_j) j_t (\Ij{t}) + (\lambda_j) J_t (\Ij{t}) $$


A primeira parte da função valor de j é caso ele venha a propor:


$$ J_t (\Ij{t}) = \max \bigg\{ L_{jt}, V_{t+1} \ej{t}{\theta_{j, t+1} }  - \ej{t}{W_{s, t+1}  }, \ej{t}{W_{j,t+1} } \bigg\} $$


A segunda parte da função valor é se se for chamado a responder:
    
$j_t (\Ij{t}) = $
    $ \ejb{t}{ Prob ({s liquidar})  \maximo{L_{j,t}, V_{t+1} \ej{t}{\theta_{j,t+1}  } - L_{s, t} } } $
    $ + \ejb{t}{ Prob ({j reorganizar})   \maximo{\es{t}{W_{j, t+1} }, \ej{t}{W_{j, t+1} } }  }$
 
 


### Como montar matrizes não funcionou, vou tentar resolver usando a forma recursiva

Sabemos que a função valor hoje é função dos parâmetros hoje e dela própria no período seguinte.

E sabemos que o update do lower bound das habilidades requer que calculemos uma função valor de acordo com um cutoff estipulado. 

Assim, vou implementar os passos a seguir:

1. Pensar a estrutura recursiva para a função valor

2. Criar uma regra de update do lower bound

3. Escrever a função valor na forma recursiva


### Escrever a função valor na forma recursiva

Agora que temos as estruturas para a forma recursiva, vamos escrever as funções na forma recursiva

Vou colar de novo todas as funções abaixo e trabalhar nelas.

In [348]:
#payoff de S quanto ele tem o direito de propor em t

#aqui eu colo Wj[t] e Ws[t] como argumentos. 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def St(teta_st, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje pois s revela esta quando propõe
    
    ls_next = teta_st
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
        
    while (Wj_val(ljt, lst, ljt, t+1) - Wj_val(cjt, ls_next, cjt, t+1) > tol and flag < 10000):
        cjt = cjt + 0.1
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    #esperança da própria habilidade no próximo período
    θs_next = expec(teta_st, t, (t+1))
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    St = [Ls(t), V[t] * θs_next - Wj_val(lst, lst, ljt, t+1), Ws_val(θs_next, ls_next, lj_next, t+1)]
    
    policy_St = np.argmax(St)
    
    
    
    return (St[policy_St])


    

    


Na probabilidade de liquidar no período seguinte, temos a seguinte expressão:

$ Prob \bigg( L_{j,t} > \maximo{V_{t+1} \es{t}{\theta_{j,t+1} }  - \es{t}{ \ej{t}{ W_{s,t+1}  } }  ,  \es{t}{ \ej{t}{ W_{j,t+1}  } } } \bigg)$

Onde os termos dentro das expectativas iteradas são calculados com base apenas nos lower bounds de hoje.

1. Vamos considerar o caso em que a esperança de j sobre a sua própria função valor  é o máximo dos argumentos

2. Vamos calcular o threshold para o caso onde ainda pode compensar reorganizar a firma

In [349]:
#vamos criar os componentes da função st

#probabilidade de j liquidar em t, dado ljt 

#não preciso me preocupar com if t == T, pois esta condição estará nas funções valor Ws e Wj

def Prob_s(teta_st, lst, ljt, t):
    
    #caso onde a esperança de j sobre a própria função é o máximo dos argumentos
    if (Wj_val(ljt, lst, ljt, t+1) > V[t] * 1 - Ws_val(lst, lst, ljt, t+1) ):
        
        #se a esperança da função valor amanhã for maior ou igual ao valor de liquidar,
        #a chance de liquidar é zero
        
        if(Wj_val(ljt, lst, ljt, t+1) >= Lj(t)):
            return 0
        else: 
            return 1
        
    #caso onde ainda compensa reorganizar a firma
    else:

        #threshold para j tentar liquidar
        
        #DÚVIDA SE AQUI EU PRECISO USAR LJ_NEXT E LS_NEXT.
        #ACHO QUE SIM, VOU CONFERIR DEPOIS
        ϕjt = ( Lj(t) + Ws_val(lst, ljt, ljt, t+1) ) / V[t]

        Prob_st = 1 - ((1 - ϕjt)**β)/((1 - ljt)**β)

        #aqui tem que ser no mínimo a igual a zero, apenas por segurança

        Prob_st = max(0, Prob_st)

        return Prob_st



In [350]:
#payoff caso s responda à uma proposta de liquidação de j


def s_liq(teta_st, lst, ljt, t):
    
    #update dos lower bounds não é necessário aqui
    #é a unica parte da função que independe das funções valor no período seguinte
    
    
    
    st_liq = [Ls(t), V[t] * expec(teta_st, t, (t+1)) - Lj(t) ]

    policy_st_liq = np.argmax(st_liq)


    st_liq = st_liq[policy_st_liq]

    return (st_liq)




In [351]:
#payoff caso s responda à uma proposta de reorganização de j

def s_reorg(teta_st, lst, ljt, t):
    
    #update dos lower bounds
    
    
    #neste caso quem revela a informação é j,
    #vamos ter que colocar o update do lower bound na estrutura da barganha do jogo depois
    lj_next = expec(ljt, t, (t+1))
    
    #algoritmo para calcular cutoff
    
    cst = lst
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
        
    while (Ws_val(lst, lst, ljt, t+1) - Ws_val(cst, cst, lj_next, t+1) > tol and flag < 10000):
        cst = cst + 0.1
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    ls_next = max(lst, cst)
    
    
    
    st_reorg = [Ws_val(ls_next, ls_next, lj_next, t+1), Ws_val(teta_st, ls_next, lj_next, t+1)]

    policy_st_reorg = np.argmax(st_reorg)


    st_reorg = st_reorg[policy_st_reorg]
    
    return (st_reorg)



In [352]:
def st(teta_st, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    st = (Prob_s(teta_st, lst, ljt, t)) * s_liq(teta_st, lst, ljt, t) + (1 - Prob_s(teta_st, lst, ljt, t)) * s_reorg(teta_st, lst, ljt, t)
    
    return (st)

In [355]:
#finalmente, a função valor de s no período t
#defini com subscrito _val porque o vetor Ws é argumento das funções serão passadas como argumento aqui

#havia uma confusão entre Ws função e Ws vetor

#percebi o erro com o link abaixo
#https://stackoverflow.com/questions/32446402/typeerror-function-object-is-not-subscriptable-in-python-3-4-3


def Ws_val(teta_st, lst, ljt, t):
    
    #condição para calcular o valor dela no último período
    if(t == T):
        
        return Ls(T-1)
    
    else:
        
        Wst = (1- λj) * St(teta_st, lst, ljt, t) + (λj)  * st(teta_st, lst, ljt, t)
    
        return Wst
        
    

In [354]:
#testando para t = 15



Ws_val(0.5, 0.5, 0.5, 15)

#it works!!!!




NameError: name 'Wj_val' is not defined

Fazendo o mesmo trabalho para a função de j

In [ ]:
#payoff de S quanto ele tem o direito de propor em t

#aqui eu colo Wj[t] e Ws[t] como argumentos. 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def Jt(teta_jt, lst, ljt, t):
    
    #update do lower bound:
    
    #lj_next é a habilidade dele hoje pois j revela esta quando propõe
    lj_next = teta_jt
    
    #algoritmo para calcular cutoff
    
    cst = lst
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
        
    while (Ws_val(lst, lst, ljt, t+1) - Ws_val(cst, cst, lj_next, t+1) > tol and flag < 10000):
        cst = cst + 0.1
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    ls_next = max(lst, cst)
    
    #esperança da própria habilidade no próximo período
    θj_next = expec(teta_jt, t, (t+1))
    
    
    Jt = [Lj(t), V[t] * θj_next - Ws_val(ls_next, ls_next, lj_next, t+1), Wj_val(θj_next, ls_next, lj_next, t+1)]
    
    policy_Jt = np.argmax(Jt)
    
    
    
    return (Jt[policy_Jt])




    


In [ ]:
#testando para t = 15, que já sabemos Wj e Ws

Jt(0.5, 0.5, 0.5, 15)

V[15]

Wj_val(0.5,0.5,0.5, 16)

In [ ]:
#vamos fazer algo similar ao Prob_s

#probabilidade de j liquidar em t, dado ljt 
def Prob_j(teta_jt, lst, ljt, t):
    
    #caso onde a esperança de s sobre a própria função é o máximo dos argumentos
    if (Ws_val(lst, lst, ljt, t+1) > V[t] * 1 - Wj_val(ljt, lst, ljt, t+1) ):
        
        #se a esperança da função valor amanhã for maior ou igual ao valor de liquidar,
        #a chance de liquidar é zero
        
        if(Ws_val(lst, lst, ljt, t+1) >= Ls(t)):
            return 0
        else: 
            return 1
        
    #caso onde ainda compensa reorganizar a firma
    else:

        #threshold para s tentar liquidar
        
        #DÚVIDA SE AQUI EU PRECISO USAR LJ_NEXT E LS_NEXT.
        #ACHO QUE SIM, VOU CONFERIR DEPOIS
        ϕst = ( Ls(t) + Wj_val(ljt, lst, ljt, t+1) ) / V[t]

        Prob_jt = 1 - ((1 - ϕst)**β)/((1 - lst)**β)

        #aqui tem que ser no mínimo a igual a zero, apenas por segurança

        Prob_jt = max(0, Prob_jt)

        return Prob_jt

In [ ]:
#payoff caso j responda à uma propojta de liquidação de j


def j_liq(teta_jt, lst, ljt, t):
    
    jt_liq = [Lj(t), V[t] * expec(teta_jt, t, (t+1)) - Ls(t) ]

    policy_jt_liq = np.argmax(jt_liq)


    return jt_liq[policy_jt_liq]




In [ ]:
#testando para t = 15

Prob_j(0.3, 1, 1, 15)

j_liq(0.3, 1, 1, 15)

In [ ]:
#payoff caso s responda à uma proposta de reorganização de j

def j_reorg(teta_jt, lst, ljt, t):
    
    jt_reorg = [Wj[t], Wj[t]]

    policy_jt_reorg = np.argmax(jt_reorg)


    jt_reorg = jt_reorg[policy_jt_reorg]
    
    return (jt_reorg)



In [ ]:
#payoff caso j responda à uma proposta de reorganização de s

def j_reorg(teta_jt, lst, ljt, t):
    
    #update dos lower bounds
    
    
    #neste caso quem revela a informação é j,
    #vamos ter que colocar o update do lower bound na estrutura da barganha do jogo depois
    ls_next = expec(lst, t, (t+1))
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
        
    while (Wj_val(ljt, lst, ljt, t+1) - Wj_val(cjt, ls_next, cjt, t+1) > tol and flag < 10000):
        cjt = cjt + 0.1
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    #esperança da própria habilidade no próximo período
    θj_next = expec(teta_jt, t, (t+1))
    
    
    
    jt_reorg = [Wj_val(lj_next, ls_next, lj_next, t+1), Wj_val(θj_next, ls_next, lj_next, t+1)]

    policy_jt_reorg = np.argmax(jt_reorg)


    jt_reorg = jt_reorg[policy_jt_reorg]
    
    return (jt_reorg)


In [ ]:
#testando para t = 15

j_reorg(0.6, 0.5, 0.5, 15)

#de fato a habilidade não muda s_reorg em t = 15 porque o próximo período é de liquidação
#nos outros períodos isso deve mudar

In [ ]:
def jt(teta_jt, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    jt = (Prob_j(teta_jt, lst, ljt, t)) * j_liq(teta_jt, lst, ljt, t) + (1 - Prob_j(teta_jt, lst, ljt, t)) * j_reorg(teta_jt, lst, ljt, t)
    
    return (jt)

In [ ]:
#testando para t = 15

jt(0.6, 0.5, 0.5, 15)



In [ ]:
#finalmente, a função valor de s no período t
#defini com subscrito _val porque o vetor Ws é argumento das funções serão passadas como argumento aqui

#havia uma confusão entre Ws função e Ws vetor

#percebi o erro com o link abaixo
#https://stackoverflow.com/questions/32446402/typeerror-function-object-is-not-subscriptable-in-python-3-4-3


def Wj_val(teta_jt, lst, ljt, t):
    
    #condição para calcular o valor dela no último período
    if(t == T):
        
        return Lj(T-1)
    
    else:
        
        
        Wjt = (1- λj) * jt(teta_jt, lst, ljt, t) +  (λj) * Jt(teta_jt, lst, ljt, t)
    
        return Wjt
        
        
        
    
    
  

In [ ]:
#testando para t = 15



Wj_val(0.5, 0.5, 0.5, 15)

#it works!!!!




In [ ]:
#loop e gráfico

#não funfou, ficou rodando mais de uma hora e não convergiu mesmo sendo 4 períodos


# #criando um vetor para guardar os resultados
# Ws_valores = []

# #range em contagem regressiva. Coloquei 4 para não demorar tanto
# for t in range(4, 0, -1):
#     Ws_hoje = Ws_val(0.5, 0.5, 0.5, t)
    
#     Ws_valores.append(Ws_hoje)
    

# #truque para reverter a ordem no eixo x
# df = Ws_valores[::-1]

# plt.plot(df)
# plt.show()



# Próximos passos



* HIGIENIZAR O CÓDIGO, DEIXAR ORGANIZADO!

* criar função de esperança baseada na função beta




* discretizar a beta para obter a esperança das funções valor no período seguinte



* ~fazer Js_val~

* ~guardar resultados de Js_val e Ws_val do último período numa  matriz 4D~

* ~pensar numa função Js_next para achar o valor de Js no próximo período baseado nos parâmetros deste período~
    * regra para atualização de lst
    * regra para atualização de ljt
    * expectativa da habilidade no período seguinte

* ~pensar na forma recursiva do jogo~
    * acho que as funções probabilidade também têm que considerar o update do lower bound, pois elas fazem parte do cenário onde o jogador não propõe


* checar todas as funções da forma recursiva
    * ver se fazem sentido
    * ver se o cálculo está correto

* considerar a estrutura de negociação do jogo

* aumentar a precisão das estimativas do jogo
    * algoritmo para cálculo do cutoff pode ter mais casas decimais
    


* passos finais
    * replicar os gráficos do artigo de referência


### Função para tirar um draw da distribuição Beta


Vamos usar o método da amostragem da inversa da CDF (https://en.wikipedia.org/wiki/Inverse_transform_sampling_method). Outra referência que usei foi: https://blogs.sas.com/content/iml/2013/07/22/the-inverse-cdf-method.html#:~:text=The%20exponential%20distribution%20has%20probability,log(1%E2%80%93u).

A CDF da Beta é 

$$ F_{\beta} ( \theta_{t+1} \mid \theta_{t} ) = 1 - \frac{ (1 - \theta_{t+1})^\beta}{ (1 - \theta_{t})^\beta }, \, \, \theta_{t} \leq \theta_{t+1} \leq 1, \, \beta \geq 1$$

Para invertê-la, basta procurarmos o valor de x tal que $F(x) = u$, onde u é uma retirada da distribuição Uniforme(0,1).

Fazendo os cálculos, esse valor de x é (ou $\theta_{t+1}$, no caso)


$$ \theta_{t+1} =  1 - exp \bigg\{ \frac{1}{\beta} \big[  log (1 - u) + \beta * log(1 - \theta_{t}) \big] \bigg\} $$


In [398]:
#código para tirar draw da distribuição beta

def draw_beta(info_hoje, β):
    
    
    
    u = uniform(0, 1)
    x = 1 - math.exp( (1/β) * (math.log(1-u) + β*math.log(1-info_hoje)) )
          
    return x




#testando

draw_beta(0.5, 1)

0.7521263237007758

Função para tirar o valor esperado do teta amanhã, dada a informação hoje.

É apenas a média de vários draws da função beta

In [367]:
def expec_beta(info_hoje, β):
    

    #para calcular o valor esperado UM PERÍODO À FRENTE, vamos tirar 1000 draws disso e fazer a média
    #se estiver correto, expec_beta(0.5, 1) deve ser próximo de 0.75, pois Beta = 1 é Uniforme
    beta_vals = []

    for t in range(1000):
        beta_amanha = draw_beta(info_hoje, β)

        beta_vals.append(beta_amanha)
        
    return round(np.mean(beta_vals),2)



In [365]:
expec_beta(0.5,1)

0.75

Como calcular a probabilidade de cada valor de teta amanhã dado teta hoje?

Fazer um binning das variáveis

RD!

In [366]:
#binning da pdf beta

#gerando 1000 draws

def bin(info_hoje, β, ndraws):
    
    beta_vals = []

    for k in range(ndraws):
        beta_amanha = draw_beta(info_hoje, β)

        beta_vals.append(beta_amanha)
    
    #conta cria os bins e conta quantos valores estão dentro deles
    teta_bins = np.zeros(len(θj_vals))  
    
    for t in range(len(teta_bins)-1):
        
        pre = (θj_vals[t-1]+θj_vals[t])/2
               
        pos = (θj_vals[t]+θj_vals[t+1])/2
        
        if(t==0):
            pre = 0
        else:
            if(t==len(teta_bins)-1):
                pos = 1
                
       
        
        
        for b in beta_vals:
            
            if(b > pre and b < pos):
                
                teta_bins[t] += 1
    
    return teta_bins/ndraws
        
        
        
        
#dá algum erro porque a soma não retorna 1 quando Beta = 1
sum(bin(0.5, 1, 1000))



0.9920000000000007

In [361]:
#vamos gerar uma matriz com 100 colunas
#cada coluna vai representar as probabilidades de teta_amanhã dado teta hoje
#assim, a coluna 1 tem as probabilidades de teta_amanhã dado que teta_hoje é 0.01


#probability mass function
pmf = np.zeros((100,100))

#exemplo para ilustrar
# pmf[0,] = bin(θj_vals[0], β, 1000)


#populando a pmf:

for t in range(len(θj_vals)-1):
    
    pmf[:,t] = bin(θj_vals[t], β, 1000)



Como obter o valor esperado da função valor no período seguinte?

Multiplicando a coluna correta de pmf pela coluna correta de Ws_array




In [363]:


sum(pmf[:,99])

0.0

In [125]:
beta_vals = []

info_hoje = 0.5
β = 1

for t in range(1000):
    beta_amanha = draw_beta(info_hoje, β)

    beta_vals.append(beta_amanha)

### RASCUNHO: Pensar a estrutura recursiva para a função valor


Vai ser um loop feito de trás para frente.

Se estivermos no último período, ele vai considerar a função valor como o valor de liquidação

Se estivermos no penúltimo período, ele vai calcular a função valor com base nas informações de hoje e no valor de continuação amanhã

De maneira geral, em qualquer período exceto o último, ele vai calcular a função valor com base nas informações de hoje e no valor de continuação amanhã


Vou começar a ensaiar a estrutura básica pensando numa função f() qualquer. Ela precisa ser avaliada entre $t \in \{1, ..., T\}$. O valor dela em T já é definido, enquanto o valor dela nos demais períodos anteriores dependem do valor dela mesma um período adiante.

$f(\theta_t, t) = f(\theta_{t+1}, t+1)$


O objetivo é criar uma função f() capaz de calcular todos os valores entre 1 e T usando apenas um conjunto de informações do período atual e o valor dela no último período.

In [ ]:
#função básica f
    
def f(θt, t):
   
    #condição para calcular o valor dela no último período
    if(t == T):
        
        return Ls(T-1)
    
    else:
        
        #regra simples de update da habilidade
        θ_next = θt + 0.1
        
        
        #retorna o máximo entre a função hoje ou ela amanhã
        return max(V[t] * θt, f(θ_next, t+1))
        
    

In [ ]:
f(0.5, 15)


In [ ]:
#criando um vetor para guardar os resultados
f_vals = []

#range em contagem regressiva
for t in range(T, 0, -1):
    f_hoje = f(0.5, t)
    
    f_vals.append(f_hoje)
    
f_vals

#truque para reverter a ordem no eixo x
df = f_vals[::-1]

plt.plot(df)
plt.show()

### Update do lower bound




In [ ]:
#ok, agora que a função básica está ok, vou tentar criar a regra de update da habilidade. 

#vou começar com uma versão levemente alterada da função f, chamada g, onde o update também contém f(t+1)

#isso porque o cutoff é função da função valor nos períodos seguintes


def g(θt, t):
   
    #condição para calcular o valor dela no último período
    if(t == T):
        
        return Ls(T-1)
    
    else:
        
        #regra mais elaborada de update da habilidade
        
        #cst é cutoff, começando com habilidade hoje
        cst = θt
        #flag é parar parar o loop se demorar muito
        flag = 1
        tol = 0.01
        #vou usar f() porque a habilidade em f() cresce 0.1 por período
        #se cst crescer 0.1, significa que a função funcionou bem
        
        while (f(θt, t+1) - g(cst, t+1) > tol and flag < 200):
            cst = cst + 0.1
            flag = flag + 1
        
        
        
        
        θ_next = cst
        
        
        #retorna o máximo entre a função hoje ou ela amanhã
        return max(V[t] * θt, g(θ_next, t+1))
        

# a função g2 retorna o valor da habilidade amanhã        
def g2(θt, t):
   
    #condição para calcular o valor dela no último período
    if(t == T):
        
        return Ls(T-1)
    
    else:
        
        #regra mais elaborada de update da habilidade
        
        #cst é cutoff, começando com habilidade hoje
        cst = θt
        #flag é parar parar o loop se demorar muito
        flag = 1
        tol = 0.01
        #vou usar f() porque a habilidade em f() cresce 0.1 por período
        #se cst crescer 0.1, significa que a função funcionou bem
        
        while (f(θt, t+1) - g(cst, t+1) > tol and flag < 1000):
            cst = cst + 0.1
            flag = flag + 1
        
        
        
        
        θ_next = cst
        
        
        #retorna o máximo entre a função hoje ou ela amanhã
        return cst
    
#f2 também retorna o valor da habilidade amanhã

def f2(θt, t):
   
    #condição para calcular o valor dela no último período
    if(t == T):
        
        return Ls(T-1)
    
    else:
        
        #regra simples de update da habilidade
        θ_next = θt + 0.1
        
        
        #retorna o máximo entre a função hoje ou ela amanhã
        return θ_next
        



In [ ]:
#testando

g(0.5, 13)


#ótimo, a habilidade cst realmente acertou a regra de aumento da habilidade de f().
g2(0.5, 13)
f2(0.5, 13)